# RSLC to INSAR
- This notebook converts RSLC data to GUNW products by running ISCE3's `insar.py`.
- Uses the `isce3_src` kernel (created using [Create_Environments.ipynb](https://github.com/isce-framework/sds-ondemand/blob/main/environments/Create_Environments.ipynb)).
- Can be ran locally (preferably on a GPU instance), or as a PCM job.

# Parameters Cell
This cell is marked `parameters`, indicating the variables within can substituted when running this notebook via `papermill`.
- `rslc_1`: S3 url to the 1st RSLC to process.
- `rslc_2`: S3 url to the 2nd RSLC to process.
- `dem_s3_url`: S3 url to the DEM file to download.
- `watermask_s3_url`: S3 url to the DEM file to download.
- `gpu_enabled`: `1` to run using the GPU, `0` to use CPU instead. **Keep in mind that while disabling the GPU processing allows this notebook to be ran on an instance without a GPU, that does not mean the instance is a non-GPU instance.** To run on a non-GPU instance on PCM, submit the job to a CPU-only queue.
- `infer_nisar_fname`: `1` to rename the final products to their NISAR filename, `0` to leave it as a concatenation of the two RSLC filenames.
- `insar_config`: The runconfig passed to `insar.py`.

### Upload parameters (PCM only)
- `timestamp`: A time-string of the format `%Y%m%dT%H%M%S` indicating the time at which the job was submitted. This helps `pcm.py` find where the results of this job will be submitted when this notebook is ran as a PCM job.

In [ ]:
rslc_1 = 's3://nisar-st-data-ondemand/ALOS-1-data/RSLC/ALPSRP118456700-L1.0.h5' # string
rslc_2 = 's3://nisar-st-data-ondemand/ALOS-1-data/RSLC/ALPSRP224230660-L1.0.h5' # string
dem_s3_url = 's3://nisar-st-data-ondemand/DEM-static/dem_Chile.tiff' # string
watermask_s3_url = '' # string
gpu_enabled = 1 # boolean
infer_nisar_fname = 1 # boolean
insar_config = '' # string
timestamp = '20240131T0123456' # string

# hysds specifications
_time_limit = 172800
_soft_time_limit = 172800
_disk_usage = '60GB'
_submission_type = 'iteration'
_label = 'RSLC to INSAR PGE'

### Pre-processing of the Parameters to convert numbers or words into `boolean` True and False values.

In [ ]:
# Convert boolean parameters because they must be specified as strings
try:
    if not isinstance(gpu_enabled, bool):
        gpu_enabled = int(gpu_enabled) > 0
except ValueError:
    if instance(gpu_enabled, str):
        gpu_enabled = gpu_enabled.lower() == 'true'
    else:
        gpu_enabled = False
print(f'{gpu_enabled=}')

try:
    if not isinstance(infer_nisar_fname, bool):
        infer_nisar_fname = int(infer_nisar_fname) > 0
except ValueError:
    if instance(infer_nisar_fname, str):
        infer_nisar_fname = infer_nisar_fname.lower() == 'true'
    else:
        infer_nisar_fname = False
print(f'{infer_nisar_fname=}')

# Extraneous parameters
focus_config = '' # string

# Functions for loading runconfig files and downloading from S3 buckets

In [ ]:
import os
import yaml
import asf_search as asf
import boto3
import aws_uploader

WORKING_DIR = os.getcwd()
HOME_DIR = os.environ['HOME']
NOTEBOOK_PGE_DIR = os.environ.get('NOTEBOOK_PGE_DIR', WORKING_DIR)
ISCE3_BUILD_DIR = os.environ.get('ISCE3_BUILD_DIR', f'{HOME_DIR}/isce3/build')

DOWNLOAD_DIR = os.path.join(WORKING_DIR, 'downloads')
EXTRACT_DIR = os.path.join(WORKING_DIR, 'alos_data')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'output')
PRODUCT_DIR = os.path.join(WORKING_DIR, 'product_path')

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PRODUCT_DIR, exist_ok=True)

if focus_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/focus.yaml'), 'r') as f:
        FOCUS_YML = yaml.safe_load(f)
else:
    print('Using custom focus.py run config...')
    FOCUS_YML = yaml.safe_load(focus_config)

if insar_config == '':
    with open(os.path.join(NOTEBOOK_PGE_DIR, '../templates/insar.yaml'), 'r') as f:
        INSAR_YML = yaml.safe_load(f)
else:
    print('Using custom insar.py run config...')
    INSAR_YML = yaml.safe_load(insar_config)

print(WORKING_DIR)

# Run ISCE3 Python Scripts
This cell runs the python scripts:
- `insar.py`: Converts RSLC -> GUNW

In [ ]:
from urllib.parse import urlparse
import isce3_config

# Download the RSLCs from S3
dest = []
for link in [rslc_1, rslc_2]:
    download_f = os.path.join(DOWNLOAD_DIR, os.path.basename(urlparse(link).path))
    if not os.path.exists(download_f):
        aws_uploader.AWS.download_s3(link, download_f)
        if os.path.exists(download_f):
            print(f'Downloaded: {download_f}')
        else:
            print(f'==========DOWNLOAD FAILED: {download_f}==========')
    else:
        print(f'{download_f} already exists, now continuing...')
    dest.append(download_f)

# Grab the bounding box for the DEM and water mask
bb1 = isce3_config.h5parse.get_bounds(dest[0])
bb2 = isce3_config.h5parse.get_bounds(dest[1])
bbox = isce3_config.BoundingBox(west=min(bb1.west, bb2.west),
                                south=min(bb1.south, bb2.south),
                                east=max(bb1.east, bb2.east),
                                north=max(bb1.north, bb2.north))

print('Union of Bounding Boxes (Auto Determined, Used if no DEM/Watermask Specified):')
print(bbox)

# Download the DEM locally
if isinstance(dem_s3_url, str) and dem_s3_url != '':
    dem_f = isce3_config.download_dem(dem_s3_url, DOWNLOAD_DIR)
else:
    print(f'DEM input ({dem_s3_url}) was not specified, downloading DEM via stage_dem.py instead.')

    dem_vrt_f = os.path.join(PRODUCT_DIR, 'dem.vrt')
    dem_f = os.path.join(PRODUCT_DIR, 'dem_0.tiff')

    # Stage the DEM using the combined bounds for dem_tiff_f
    _cmd_stage_dem = f'mamba run -n isce3_src --live-stream python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_dem.py --margin=10 -b {bbox.west} {bbox.south} {bbox.east} {bbox.north} -o {dem_vrt_f}'
    print(f'Executing:\n    {_cmd_stage_dem}')
    !{_cmd_stage_dem}

# Download the Watermask locally
if isinstance(watermask_s3_url, str) and watermask_s3_url != '':
    watermask_f = isce3_config.download_dem(watermask_s3_url, DOWNLOAD_DIR)
else:
    print(f'DEM input ({watermask_s3_url}) was not specified, downloading DEM via stage_watermask.py instead.')

    watermask_vrt_f = os.path.join(PRODUCT_DIR, 'WATERMASK.vrt')
    watermask_f = os.path.join(PRODUCT_DIR, 'WATERMASK_0.tiff')

    # Stage the DEM using the combined bounds for watermask_tiff_f
    _cmd_stage_watermask = f'mamba run -n isce3_src --live-stream python {ISCE3_BUILD_DIR}/packages/nisar/workflows/stage_watermask.py --margin=10 -b {bbox.west} {bbox.south} {bbox.east} {bbox.north} -o {watermask_vrt_f}'
    print(f'Executing:\n    {_cmd_stage_watermask}')
    !{_cmd_stage_watermask}

# Run INSAR
b1 = os.path.basename(os.path.splitext(dest[0])[0])
b2 = os.path.basename(os.path.splitext(dest[1])[0])
yml_path = os.path.join(OUTPUT_DIR, 'insar_final.yml')
output_f = os.path.join(PRODUCT_DIR, f'{b1}_{b2}.h5')
isce3_config.write_insar_config(INSAR_YML,
                                dest[0],
                                dest[1],
                                dem_f,
                                watermask_f,
                                yml_path,
                                gpu_enabled,
                                output_f)
print(f'{os.path.exists(yml_path)=}')
print(f'Now running insar.py to generate {output_f}...')
print(f'Executing:\n    mamba run --live-stream -n isce3_src python {ISCE3_BUILD_DIR}/packages/nisar/workflows/insar.py {yml_path}')
!mamba run --no-capture-output -p /home/jovyan/test/.conda python {ISCE3_BUILD_DIR}/packages/nisar/workflows/insar.py {yml_path}

# Automatic Stage Out for PCM Jobs (r4.0.0)
This routine creates a folder which matches the regex:

#### GUNW
**Example:** `NISAR_L2_PR_GUNW_000_000_Z_000_000_0000_HH_20240131T000000_20250131T000000_20250131T000001_20250131T000002_0A1B2C_A_F_Z_000`
```
(?P<id>NISAR_L2_(?P<processing_type>PR)_GUNW_(?P<reference_cycle_number>\d{3})_(?P<relative_orbit_number>\d{3})_(?P<Direction>\w)_(?P<TrackFrame>\d{3})_(?P<secondary_cycle_number>\d{3})_(?P<radar_processing_mode>\d{4})_(?P<Polarization>\w{2})_(?P<RefRadarStartDateTime>(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})T\d{6})_(?P<RefRadarStopDateTime>\d{8}T\d{6})_(?P<SecRadarStartDateTime>\d{8}T\d{6})_(?P<SecRadarStopDateTime>\d{8}T\d{6})_(?P<composite_release_id>\w{6})_(?P<Fidelity>\w)_(?P<CoverageIndicator>[F|P])_(?P<processing_center>\w)_(?P<product_counter>\d{3}))$
```

This enables PCM to automatically upload the results of this notebook to the `s3://nisar-{system_name}-rs-ondemand` bucket.

`system_name` is one of the following:
- `st` (for Science Team)
- `adt` (for ADT)
- `iot` (for IOT)

In [ ]:
import h5py
import json

# Grab some appropriate values for PCM to automatically stage out
polarization = 'HH'
regex_name = isce3_config.GUNW_FORMAT.format(
    polarization=polarization,
    timestamp=timestamp)
auto_output_dir = os.path.join(WORKING_DIR, regex_name)
product_suffix = os.path.basename(output_f)
product_base_fname = os.path.splitext(product_suffix)[0]

# Create the automatic stage-out directory
!mkdir -p {auto_output_dir}
!cp {WORKING_DIR}/*-output.ipynb {auto_output_dir}
!cp {WORKING_DIR}/*.txt {auto_output_dir}
!cp {WORKING_DIR}/*.json {auto_output_dir}
!mv {yml_path} {auto_output_dir}
!touch {auto_output_dir}/{product_base_fname}.txt

# Move the products
infer_failed = False
if infer_nisar_fname:
    print('Now inferring NISAR filename for INSAR products...')
    infer_list = []
    try:
        for iter_fname in os.scandir(PRODUCT_DIR):
            if iter_fname.is_file() and iter_fname.path.endswith(product_suffix):
                inferred_prod_name = isce3_config.h5parse.infer_nisar_name(iter_fname.path)
                print(f'Inferred NISAR filename:\n    - {iter_fname.path} -> {inferred_prod_name}')
                infer_list.append([iter_fname.path, os.path.join(auto_output_dir, inferred_prod_name)])
        for insar_path_pair in infer_list:
            print(f'Now moving:\n    - {insar_path_pair[0]} -> {insar_path_pair[1]}')
            os.rename(insar_path_pair[0], insar_path_pair[1])
    except Exception as e:
        print('An error occured while trying to infer a NISAR filename. Using standard name instead.')
        print(e)
        infer_failed = True

if not infer_nisar_fname or infer_failed:
    #!mv {PRODUCT_DIR}/*{product_suffix} {auto_output_dir}
    print(f'mv {PRODUCT_DIR}/*{product_suffix} {auto_output_dir}')

# Move the new path variables to avoid confusion
yml_path = os.path.join(auto_output_dir, os.path.basename(yml_path))
output_f = os.path.join(auto_output_dir, os.path.basename(output_f))

# Create metadata files for the automatic stage-out directory
with open(os.path.join(auto_output_dir, f'{regex_name}.met.json'), 'w', encoding='utf-8') as f:
    content = {
        'polarization': polarization,
    }
    f.write(json.dumps(content))
with open(os.path.join(auto_output_dir, f'{regex_name}.dataset.json'), 'w', encoding='utf-8') as f:
    content = {
         'version': 'v1.0',
         'label': 'This is purely an EXAMPLE metadata file, the values in this file are not representative of this product.',
         'location': {
           'type': 'polygon',
           'coordinates': [
             [
                [-122.9059682940358,40.47090915967475],
                [-121.6679748715316,37.84406528996276],
                [-120.7310161872557,38.28728069813177],
                [-121.7043611684245,39.94137004454238],
                [-121.9536916840953,40.67097860759095],
                [-122.3100379696548,40.7267890636145],
                [-122.7640648263371,40.5457010812299],
                [-122.9059682940358,40.47090915967475]
              ]
            ]
        },
        'starttime': '2017-01-01T00:00:00',
        'endtime': '2017-01-01T00:05:00',
    }
    f.write(json.dumps(content))